#### This notebook will find the  average one year value of variables in CONUS where the CAMS data is regridded from 0.75 X 0.75 to a finer grid

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import xarray as xr
import regionmask

In [2]:
data = xr.open_dataset("data.nc")
data

<xarray.Dataset> Size: 7MB
Dimensions:        (valid_time: 12, latitude: 35, longitude: 81)
Coordinates:
  * valid_time     (valid_time) datetime64[ns] 96B 2010-01-01 ... 2010-12-01
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
Data variables: (12/48)
    d2m            (valid_time, latitude, longitude) float32 136kB ...
    t2m            (valid_time, latitude, longitude) float32 136kB ...
    bcaod550       (valid_time, latitude, longitude) float32 136kB ...
    chnk           (valid_time, latitude, longitude) float32 136kB ...
    duaod550       (valid_time, latitude, longitude) float32 136kB ...
    lai_hv         (valid_time, latitude, longitude) float32 136kB ...
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 136kB ...
    aermsssss      (valid_time, latitude, longitude) float32 136kB ...
    aermssssm      (valid_time, latitude, longitude) float32 136kB ...
    aermssssl      (valid_time, latitude, longitude) float32 136kB ...
    aermsssu       (valid_time, latitude, longitude) float32 136kB ...
    aermssso2      (valid_time, latitude, longitude) float32 136kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
## resample the data to 1 year average

year_avg = data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 545kB
Dimensions:        (valid_time: 1, latitude: 35, longitude: 81)
Coordinates:
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
  * valid_time     (valid_time) datetime64[ns] 8B 2010-12-31
Data variables: (12/48)
    d2m            (valid_time, latitude, longitude) float32 11kB 278.3 ... 2...
    t2m            (valid_time, latitude, longitude) float32 11kB 281.6 ... 2...
    bcaod550       (valid_time, latitude, longitude) float32 11kB 0.006512 .....
    chnk           (valid_time, latitude, longitude) float32 11kB 0.01415 ......
    duaod550       (valid_time, latitude, longitude) float32 11kB 0.001253 .....
    lai_hv         (valid_time, latitude, longitude) float32 11kB 3.752 ... 0.0
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 11kB 9.168e-06 ....
    aermsssss      (valid_time, latitude, longitude) float32 11kB 6.539e-07 ....
    aermssssm      (valid_time, latitude, longitude) float32 11kB 4.983e-05 ....
    aermssssl      (valid_time, latitude, longitude) float32 11kB 1.505e-05 ....
    aermsssu       (valid_time, latitude, longitude) float32 11kB 2.849e-06 ....
    aermssso2      (valid_time, latitude, longitude) float32 11kB 6.057e-07 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
## delete variables that have NaN in them

vars_with_nan = [var for var in year_avg if year_avg[var].isnull().any()]
year_avg= year_avg.drop_vars(vars_with_nan)
year_avg

<xarray.Dataset> Size: 523kB
Dimensions:        (valid_time: 1, latitude: 35, longitude: 81)
Coordinates:
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
  * valid_time     (valid_time) datetime64[ns] 8B 2010-12-31
Data variables: (12/46)
    d2m            (valid_time, latitude, longitude) float32 11kB 278.3 ... 2...
    t2m            (valid_time, latitude, longitude) float32 11kB 281.6 ... 2...
    bcaod550       (valid_time, latitude, longitude) float32 11kB 0.006512 .....
    chnk           (valid_time, latitude, longitude) float32 11kB 0.01415 ......
    duaod550       (valid_time, latitude, longitude) float32 11kB 0.001253 .....
    lai_hv         (valid_time, latitude, longitude) float32 11kB 3.752 ... 0.0
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 11kB 9.168e-06 ....
    aermsssss      (valid_time, latitude, longitude) float32 11kB 6.539e-07 ....
    aermssssm      (valid_time, latitude, longitude) float32 11kB 4.983e-05 ....
    aermssssl      (valid_time, latitude, longitude) float32 11kB 1.505e-05 ....
    aermsssu       (valid_time, latitude, longitude) float32 11kB 2.849e-06 ....
    aermssso2      (valid_time, latitude, longitude) float32 11kB 6.057e-07 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
## note that even though the time dimesion, essentially shows 2010-12-31,
## what it actually means is yearly average

year_avg.to_dataframe()

d2m         t2m  bcaod550      chnk  \
valid_time latitude longitude                                               
2010-12-31 49.5     -126.00    278.348907  281.556824  0.006512  0.014148   
                    -125.25    279.088837  282.437836  0.006916  0.013934   
                    -124.50    279.470001  282.923370  0.007213  0.013523   
                    -123.75    279.041290  282.562347  0.007478  0.013581   
                    -123.00    277.851837  281.525085  0.008017  0.014813   
...                                   ...         ...       ...       ...   
           24.0     -69.00     293.523407  297.542999  0.004872  0.012293   
                    -68.25     293.495728  297.488953  0.004856  0.012303   
                    -67.50     293.465454  297.455261  0.004833  0.012302   
                    -66.75     293.443970  297.427094  0.004824  0.012318   
                    -66.00     293.438751  297.397156  0.004825  0.012332   

                               duaod550    lai_hv    lai_lv            msl  \
valid_time latitude longitude                                                
2010-12-31 49.5     -126.00    0.001253  3.752065  2.928701  101390.726562   
                    -125.25    0.001246  4.228424  1.811178  101402.250000   
                    -124.50    0.001238  2.194234  0.692861  101427.570312   
                    -123.75    0.001204  0.905955  0.624898  101459.429688   
                    -123.00    0.001140  3.628265  2.724324  101485.656250   
...                                 ...       ...       ...            ...   
           24.0     -69.00     0.024328  0.000000  0.000000  101571.789062   
                    -68.25     0.024477  0.000000  0.000000  101578.000000   
                    -67.50     0.024507  0.000000  0.000000  101583.164062   
                    -66.75     0.024585  0.000000  0.000000  101590.195312   
                    -66.00     0.024893  0.000000  0.000000  101595.625000   

                               omaod550         pm2p5  ...     aermssdul  \
valid_time latitude longitude                          ...                 
2010-12-31 49.5     -126.00    0.055563  5.288607e-09  ...  5.697568e-07   
                    -125.25    0.060980  6.618710e-09  ...  5.552510e-07   
                    -124.50    0.063776  7.792362e-09  ...  5.443322e-07   
                    -123.75    0.067136  1.080445e-08  ...  5.231616e-07   
                    -123.00    0.076543  1.931933e-08  ...  4.871432e-07   
...                                 ...           ...  ...           ...   
           24.0     -69.00     0.046011  9.519893e-09  ...  1.086019e-05   
                    -68.25     0.045661  9.399162e-09  ...  1.102058e-05   
                    -67.50     0.045540  9.288230e-09  ...  1.103588e-05   
                    -66.75     0.045691  9.213097e-09  ...  1.106749e-05   
                    -66.00     0.045851  9.209978e-09  ...  1.116341e-05   

                               aermssbchphil  aermssomhphil  aermssbchphob  \
valid_time latitude longitude                                                
2010-12-31 49.5     -126.00     1.359443e-07   2.107467e-06   5.559153e-07   
                    -125.25     1.506114e-07   2.379295e-06   5.843853e-07   
                    -124.50     1.616488e-07   2.564180e-06   6.047914e-07   
                    -123.75     1.798799e-07   2.806439e-06   6.146706e-07   
                    -123.00     2.252995e-07   3.389544e-06   6.264335e-07   
...                                      ...            ...            ...   
           24.0     -69.00      3.103107e-08   5.009322e-07   4.866120e-07   
                    -68.25      3.011268e-08   4.867323e-07   4.857583e-07   
                    -67.50      2.886006e-08   4.733052e-07   4.846137e-07   
                    -66.75      2.775320e-08   4.601205e-07   4.847639e-07   
                    -66.00      2.699588e-08   4.502798e-07   4.856791e-07   

                 

In [6]:
## lets add new data points in the longitude to convert the resolution into a smaller resolution (0.75X0.75)...
## to a finer resolution

new_lon = np.linspace(year_avg.longitude[0],
                      year_avg.longitude[-1], year_avg.sizes["longitude"] * 20)

In [7]:
## lets add new data points in the latitude to convert the dataframe into a finer grid

new_lat = np.linspace(year_avg.latitude[0], year_avg.latitude[-1],
                                                            year_avg.sizes["latitude"] * 20)

In [8]:
## find the corresponding values of the variables in the finer grid

year_avg_finer= year_avg.interp(longitude=new_lon, latitude=new_lat)
year_avg_finer

<xarray.Dataset> Size: 417MB
Dimensions:        (valid_time: 1, latitude: 700, longitude: 1620)
Coordinates:
  * valid_time     (valid_time) datetime64[ns] 8B 2010-12-31
  * longitude      (longitude) float64 13kB -126.0 -126.0 ... -66.04 -66.0
  * latitude       (latitude) float64 6kB 49.5 49.46 49.43 ... 24.07 24.04 24.0
Data variables: (12/46)
    d2m            (valid_time, latitude, longitude) float64 9MB 278.3 ... 293.4
    t2m            (valid_time, latitude, longitude) float64 9MB 281.6 ... 297.4
    bcaod550       (valid_time, latitude, longitude) float64 9MB 0.006512 ......
    chnk           (valid_time, latitude, longitude) float64 9MB 0.01415 ... ...
    duaod550       (valid_time, latitude, longitude) float64 9MB 0.001253 ......
    lai_hv         (valid_time, latitude, longitude) float64 9MB 3.752 ... 0.0
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float64 9MB 9.168e-06 .....
    aermsssss      (valid_time, latitude, longitude) float64 9MB 6.539e-07 .....
    aermssssm      (valid_time, latitude, longitude) float64 9MB 4.983e-05 .....
    aermssssl      (valid_time, latitude, longitude) float64 9MB 1.505e-05 .....
    aermsssu       (valid_time, latitude, longitude) float64 9MB 2.849e-06 .....
    aermssso2      (valid_time, latitude, longitude) float64 9MB 6.057e-07 .....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [9]:
## Lets load the county shapefile

SHAPE_PATH = os.path.join("County_shapefile",'gz_2010_us_050_00_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)

county_gdf.head(12)

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01029,01,029,Cleburne,County,560.100,"POLYGON ((-85.38872 33.91304, -85.38088 33.873..."
1,0500000US01031,01,031,Coffee,County,678.972,"POLYGON ((-86.03044 31.61894, -86.00408 31.619..."
2,0500000US01037,01,037,Coosa,County,650.926,"POLYGON ((-86.00928 33.10164, -86.00917 33.090..."
3,0500000US01039,01,039,Covington,County,1030.456,"POLYGON ((-86.34851 30.99434, -86.35023 30.994..."
4,0500000US01041,01,041,Crenshaw,County,608.840,"POLYGON ((-86.14699 31.68045, -86.14711 31.663..."
5,0500000US01045,01,045,Dale,County,561.150,"POLYGON ((-85.79043 31.32027, -85.79033 31.323..."
6,0500000US01049,01,049,DeKalb,County,777.093,"POLYGON ((-85.57593 34.82373, -85.56142 34.750..."
7,0500000US01053,01,053,Escambia,County,945.080,"POLYGON ((-87.16308 30.99904, -87.16408 30.999..."
8,0500000US01057,01,057,Fayette,County,627.660,"POLYGON ((-87.63593 33.87874, -87.63604 33.872..."
9,0500000US01061,01,061,Geneva,County,574.408,"POLYGON ((-85.77267 30.99462, -85.77966 30.994..."


In [10]:
## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATE'] + county_gdf['COUNTY']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

GEO_ID          object
STATE           object
COUNTY          object
NAME            object
LSAD            object
CENSUSAREA     float64
geometry      geometry
fips             int32
dtype: object

In [11]:
county_gdf = county_gdf.drop([ 'GEO_ID','CENSUSAREA','STATE','COUNTY','LSAD'], axis=1)
county_gdf

,NAME,geometry,fips
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029
1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031
2,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037
3,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039
4,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041
...,...,...,...
3216,San Sebastián,"POLYGON ((-66.90748 18.25314, -66.90739 18.253...",72131
3217,Santa Isabel,"POLYGON ((-66.37968 17.94398, -66.38029 17.943...",72133
3218,Toa Baja,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...",72137
3219,Trujillo Alto,"POLYGON ((-66.02917 18.37590, -66.02828 18.376...",72139


In [12]:
county_gdf.dtypes

NAME          object
geometry    geometry
fips           int32
dtype: object

In [13]:
## The life expectency data

le=pd.read_csv('LE_2010.csv')
le = le.drop([ 'Unnamed: 0'], axis=1)
le

,location_name,fips,year,MeanLifeExpectency
0,Autauga County (Alabama),1001,2010,75.728489
1,Baldwin County (Alabama),1003,2010,77.826608
2,Barbour County (Alabama),1005,2010,75.841973
3,Bibb County (Alabama),1007,2010,73.705432
4,Blount County (Alabama),1009,2010,75.753407
...,...,...,...,...
3122,Sweetwater County (Wyoming),56037,2010,77.582422
3123,Teton County (Wyoming),56039,2010,83.771012
3124,Uinta County (Wyoming),56041,2010,77.836526
3125,Washakie County (Wyoming),56043,2010,78.674180


In [14]:
## the counties that are in life expectency data but the shape file does not exist

county_noshape = le.loc[~le['fips'].isin(county_gdf['fips'])].copy()
county_noshape

,location_name,fips,year,MeanLifeExpectency
80,Chugach Census Area (Alaska),2063,2010,77.733009
81,Copper River Census Area (Alaska),2066,2010,77.733009
83,Kusilvak Census Area (Alaska),2158,2010,69.723907
85,Kobuk Census Area (Alaska),2140,2010,73.339132
87,Aleutian Islands Census Area (Alaska),2010,2010,82.631153
96,Skagway-Yakutat-Angoon Census Area (Alaska),2231,2010,80.202830
97,Skagway-Hoonah-Angoon Census Area (Alaska),2232,2010,80.202830
102,Prince of Wales-Outer Ketchikan Census Area (A...,2201,2010,78.068610
104,Wrangell-Petersburg Census Area (Alaska),2280,2010,78.068610
338,Dade County (Florida),12025,2010,80.576828


### Note that the CAMS weather data is only for CONUS, Alaska and Hawaii has not been included, which is, 38 census area of Alaska and 5 counties of Hawaii. Since, there 15 locations whose shapfile does not exist as seen above and 9 of them are of Alaska; 3127-38-5-6=3078 shapefile of the life expectency data in total exist.

In [15]:
county_list=county_gdf['fips'].tolist()

In [16]:
var=county_list[4]
print(var)
print(len(county_list))

1041
3221


In [17]:
def single_county(var):
    
    ''' Function to find the average value of weather data variables of a single county
        as listed in county_list.
         Args:
         --------
             var (str): The name of the state.

         Returns:
         --------
            final_df: Dataframe consisting of the columns in the shape file and corresponding value of the variables
                      which in this case is the d2m and t2m.
    '''
    
    county = county_gdf[county_gdf.fips.isin([var])]
    county_mask = regionmask.mask_3D_geopandas(county,year_avg_finer.longitude,year_avg_finer.latitude)
    df_masked = year_avg_finer.where(county_mask)
    summary = df_masked.groupby("region").mean(["latitude", "longitude","valid_time"])
    df2=summary.to_dataframe()   
    
    df = pd.merge(county_gdf, df2, left_index=True, right_index=True, how='left') 
    
    ## It is VERY IMPORTANT that the left dataframe that is merged is a Geopandas Dataframe
    ## Not pandas dataframe. While a Geopandas dataframe can be easily plotted, a pandas  
    ## even if it consists Geometry column, pandas will not recognize it and it will not be plotted

    final_df=df.dropna()
    return final_df

In [18]:
single_county(county_list[0])

,NAME,geometry,fips,d2m,t2m,bcaod550,chnk,duaod550,lai_hv,lai_lv,...,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029,282.519952,289.747999,0.006229,0.018,0.007863,3.534843,2.750073,...,0.000002,1.376627e-07,0.000003,5.241108e-07,0.000013,1.991006e-07,0.000013,7.276422e-07,0.000009,0.000004


In [19]:
%%time

## get the average for all of the counties

df_list=[]

for i in range(0,len(county_list)):
    try:
        df_list.append(single_county(county_list[i]))
    except:
        pass
df_list[:4] ## see first 4 df

C:\Users\Shisir\.conda\envs\Weather_Prediction\lib\site-packages\regionmask\core\_geopandas.py:275: UserWarning: No gridpoint belongs to any region. Returning an empty mask with shape (0, 700, 1620)
  mask_3D = _mask_3D(
C:\Users\Shisir\.conda\envs\Weather_Prediction\lib\site-packages\regionmask\core\_geopandas.py:275: UserWarning: No gridpoint belongs to any region. Returning an empty mask with shape (0, 700, 1620)
  mask_3D = _mask_3D(
C:\Users\Shisir\.conda\envs\Weather_Prediction\lib\site-packages\regionmask\core\_geopandas.py:275: UserWarning: No gridpoint belongs to any region. Returning an empty mask with shape (0, 700, 1620)
  mask_3D = _mask_3D(
C:\Users\Shisir\.conda\envs\Weather_Prediction\lib\site-packages\regionmask\core\_geopandas.py:275: UserWarning: No gridpoint belongs to any region. Returning an empty mask with shape (0, 700, 1620)
  mask_3D = _mask_3D(
C:\Users\Shisir\.conda\envs\Weather_Prediction\lib\site-packages\regionmask\core\_geopandas.py:275: UserWarning: No 

CPU times: total: 19min 2s
Wall time: 28min 11s


C:\Users\Shisir\.conda\envs\Weather_Prediction\lib\site-packages\regionmask\core\_geopandas.py:275: UserWarning: No gridpoint belongs to any region. Returning an empty mask with shape (0, 700, 1620)
  mask_3D = _mask_3D(


[       NAME                                           geometry  fips  \
 0  Cleburne  POLYGON ((-85.38872 33.91304, -85.38088 33.873...  1029   
 
           d2m         t2m  bcaod550   chnk  duaod550    lai_hv    lai_lv  ...  \
 0  282.519952  289.747999  0.006229  0.018  0.007863  3.534843  2.750073  ...   
 
    aermssdul  aermssbchphil  aermssomhphil  aermssbchphob  aermssomhphob  \
 0   0.000002   1.376627e-07       0.000003   5.241108e-07       0.000013   
 
       aermsssss  aermssssm     aermssssl  aermsssu  aermssso2  
 0  1.991006e-07   0.000013  7.276422e-07  0.000009   0.000004  
 
 [1 rows x 49 columns],
      NAME                                           geometry  fips  \
 1  Coffee  POLYGON ((-86.03044 31.61894, -86.00408 31.619...  1031   
 
           d2m        t2m  bcaod550      chnk  duaod550    lai_hv    lai_lv  \
 1  284.478783  291.46722  0.006493  0.017623  0.011795  3.962753  2.382034   
 
    ...  aermssdul  aermssbchphil  aermssomhphil  aermssbchphob  aerms

In [20]:
initial_df=pd.concat(df_list)
initial_df

,NAME,geometry,fips,d2m,t2m,bcaod550,chnk,duaod550,lai_hv,lai_lv,...,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029,282.519952,289.747999,0.006229,0.018000,0.007863,3.534843,2.750073,...,0.000002,1.376627e-07,0.000003,5.241108e-07,0.000013,1.991006e-07,0.000013,7.276422e-07,0.000009,3.835913e-06
1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031,284.478783,291.467220,0.006493,0.017623,0.011795,3.962753,2.382034,...,0.000004,1.420224e-07,0.000002,5.478736e-07,0.000013,2.552366e-07,0.000018,1.329914e-06,0.000008,9.663290e-07
2,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037,283.228332,290.562380,0.006220,0.018000,0.009604,3.835704,2.562044,...,0.000003,1.305378e-07,0.000003,5.302644e-07,0.000013,2.179846e-07,0.000015,8.928507e-07,0.000008,2.505399e-06
3,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039,284.965620,291.764851,0.006589,0.017145,0.012353,3.799539,2.666721,...,0.000004,1.449063e-07,0.000002,5.551711e-07,0.000013,2.633969e-07,0.000019,1.459187e-06,0.000008,9.372548e-07
4,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041,283.961277,291.405402,0.006434,0.017994,0.011639,3.992757,2.511330,...,0.000004,1.368799e-07,0.000002,5.466558e-07,0.000013,2.488846e-07,0.000017,1.239424e-06,0.000008,1.037784e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,Niobrara,"POLYGON ((-104.05298 42.85955, -104.05286 42.7...",56027,271.644102,282.251425,0.005408,0.018000,0.007856,3.193681,0.675933,...,0.000008,1.084863e-07,0.000002,4.660618e-07,0.000010,2.131302e-07,0.000013,8.465100e-07,0.000004,9.864359e-07
3139,Platte,"POLYGON ((-104.77417 42.60996, -104.76422 42.6...",56031,270.578710,281.137364,0.005056,0.018000,0.006277,4.347885,0.799343,...,0.000006,1.019221e-07,0.000002,4.352902e-07,0.000009,2.002938e-07,0.000012,7.601027e-07,0.000004,1.090682e-06
3140,Sweetwater,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",56037,268.845323,278.305063,0.004760,0.018000,0.004099,2.367600,0.499361,...,0.000004,9.303937e-08,0.000002,4.126819e-07,0.000008,2.043313e-07,0.000013,9.206248e-07,0.000003,1.313561e-06
3141,Washakie,"POLYGON ((-107.12892 43.99455, -107.12797 43.9...",56043,269.997583,279.094165,0.005379,0.018000,0.003869,4.695147,0.553820,...,0.000003,9.974446e-08,0.000002,4.714552e-07,0.000009,2.165582e-07,0.000013,9.518787e-07,0.000003,6.129192e-07


#### There must be a total of 3078 counties as mentioned above Cell 15.

In [21]:
final_df=le.merge(initial_df,how='left', on='fips')
final_df=final_df.dropna()
final_df

,location_name,fips,year,MeanLifeExpectency,NAME,geometry,d2m,t2m,bcaod550,chnk,...,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,Autauga County (Alabama),1001,2010,75.728489,Autauga,"POLYGON ((-86.52469 32.70706, -86.52443 32.707...",283.457334,291.095963,0.006285,0.018000,...,3.301108e-06,1.285237e-07,0.000003,5.391705e-07,0.000013,2.307045e-07,0.000016,1.013401e-06,0.000008,1.608836e-06
1,Baldwin County (Alabama),1003,2010,77.826608,Baldwin,"POLYGON ((-87.41247 30.57386, -87.41271 30.573...",286.774566,292.243337,0.006408,0.015251,...,4.366019e-06,1.199524e-07,0.000002,5.609072e-07,0.000013,2.893363e-07,0.000021,2.096473e-06,0.000008,1.137713e-06
2,Barbour County (Alabama),1005,2010,75.841973,Barbour,"POLYGON ((-85.13285 31.80037, -85.13283 31.798...",284.126592,291.031410,0.006434,0.018000,...,3.333076e-06,1.459408e-07,0.000002,5.376137e-07,0.000013,2.384609e-07,0.000017,1.125823e-06,0.000008,1.211679e-06
3,Bibb County (Alabama),1007,2010,73.705432,Bibb,"POLYGON ((-87.11632 32.83560, -87.15529 32.835...",283.444642,290.724175,0.006232,0.018000,...,3.198924e-06,1.257074e-07,0.000003,5.363660e-07,0.000013,2.217685e-07,0.000015,9.108023e-07,0.000008,1.981303e-06
4,Blount County (Alabama),1009,2010,75.753407,Blount,"POLYGON ((-86.73121 34.01470, -86.72710 34.016...",282.865906,289.764081,0.006107,0.018000,...,2.562438e-06,1.292587e-07,0.000003,5.195278e-07,0.000013,1.987523e-07,0.000013,7.087727e-07,0.000008,2.444648e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3122,Sweetwater County (Wyoming),56037,2010,77.582422,Sweetwater,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",268.845323,278.305063,0.004760,0.018000,...,3.759808e-06,9.303937e-08,0.000002,4.126819e-07,0.000008,2.043313e-07,0.000013,9.206248e-07,0.000003,1.313561e-06
3123,Teton County (Wyoming),56039,2010,83.771012,Teton,"POLYGON ((-111.04668 43.80830, -111.04672 43.8...",267.885333,274.408613,0.004908,0.018000,...,9.094782e-07,9.833094e-08,0.000002,4.229372e-07,0.000008,2.082300e-07,0.000013,1.041489e-06,0.000003,2.643823e-07
3124,Uinta County (Wyoming),56041,2010,77.836526,Uinta,"POLYGON ((-110.04864 41.04008, -110.04848 40.9...",270.030103,276.900243,0.004992,0.018000,...,1.629245e-06,1.027302e-07,0.000002,4.277145e-07,0.000008,2.131106e-07,0.000013,1.039654e-06,0.000003,6.127781e-07
3125,Washakie County (Wyoming),56043,2010,78.674180,Washakie,"POLYGON ((-107.12892 43.99455, -107.12797 43.9...",269.997583,279.094165,0.005379,0.018000,...,3.202959e-06,9.974446e-08,0.000002,4.714552e-07,0.000009,2.165582e-07,0.000013,9.518787e-07,0.000003,6.129192e-07
